# ooookay you motherfuckers!
lets roll!


!pip install pyspark


from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("xyi") \
    .master("local") \
    .getOrCreate()

In [1]:
# general
import pyarrow.parquet as pq
import pandas as pd
from scipy import stats
import random

# scoring
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# drawing
import seaborn as sns
import matplotlib.pyplot as plt

# transform
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

# estimators
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
import lightgbm as lgb


# spark
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [2]:
def open_df(part):
    
    import pandas as pd
    
    # open
    df = pd.read_csv('train_data/tst_'+str(part)+'.csv', header=None)
    
    # очень неудобные названия
    col_names = dict()
    for i in range(len(df.columns)):
                       col_names[df.columns[i]] = cols[i]
    df = df.rename(columns=col_names)
    return df

In [3]:
def sample_df(df3, total_rows =  100000, neg_percent = 50, pos_percent = 50):
#     print( 'sample_df3 start')
    df3_pos = df3[df3['flag'] == 1].sample(int(total_rows / 100 * pos_percent))
    df3_neg = df3[df3['flag'] == 0].sample(int(total_rows / 100 * neg_percent))
    df3_pos = df3_pos.reset_index()
    df3_neg = df3_neg.reset_index()
    df3_pos = df3_pos.drop('index', axis=1)
    df3_neg = df3_neg.drop('index', axis=1)
    df3 = pd.concat([df3_pos, df3_neg])
    
#     print( 'sample_df3 end')
#     print('-')      
#     print('-')      
    #print('-') 
    
    return df3

In [4]:
def mean(lst): 
    return sum(lst) / len(lst) 

In [5]:
cols = ['id', 'rn', 'pre_since_opened', 'pre_since_confirmed', 'pre_pterm', 'pre_fterm', 'pre_till_pclose', 'pre_till_fclose', 'pre_loans_credit_limit', 'pre_loans_next_pay_summ',
 'pre_loans_outstanding', 'pre_loans_total_overdue', 'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate', 'pre_loans5', 'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90', 'is_zero_loans5',
 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90', 'pre_util', 'pre_over2limit', 'pre_maxover2limit', 'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit', 'enc_paym_0',
 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9', 'enc_paym_10', 'enc_paym_11','enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15',
 'enc_paym_16', 'enc_paym_17', 'enc_paym_18', 'enc_paym_19', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24', 'enc_loans_account_holder_type', 'enc_loans_credit_status','enc_loans_credit_type', 'enc_loans_account_cur', 'pclose_flag',
 'fclose_flag']

In [6]:
df = open_df(0)
df2 = open_df(1)
df3 = open_df(2)

In [7]:
answers = pd.read_csv('train_target.csv')

In [8]:
df = pd.merge(df, answers, on='id', how='left')
df2 = pd.merge(df2, answers, on='id', how='left')
df3 = pd.merge(df3, answers, on='id', how='left')

In [9]:
df.loc[df.id == 495, 'rn'].shape[0]

8

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\anaconda3\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionRese

# 1 гиппотеза. Сбалансировать таргет 50/50 


In [46]:
df_balanced = sample_df(df, 100000, 50, 50)

sample_df3 start
sample_df3 end
-


In [47]:
results_dict = dict()

In [48]:
models = [
    KNeighborsClassifier(),
    SVC(),
    RandomForestClassifier(),
    MLPClassifier(),
    LogisticRegression(),
    DecisionTreeClassifier(),
    GaussianNB(),
]
#     xgb(),
#     lgb(),
# ]

In [49]:
x = df_balanced.drop('flag', axis=1)
y = df_balanced['flag']


In [50]:
for model in models:
    
    cv_scores = cross_val_score(model, x, y, cv=3, scoring='roc_auc')
    results_dict[str(model)[0:str(model).find('(')]] = mean(cv_scores)


In [59]:
results_dict

{'KNeighborsClassifier': 0.800007655519486,
 'SVC': 0.5037922246810788,
 'RandomForestClassifier': 0.6497169063555427,
 'MLPClassifier': 0.5475719430597831,
 'LogisticRegression': 0.5050264873199198,
 'DecisionTreeClassifier': 0.5506800682419088,
 'GaussianNB': 0.5812535989447697}

In [60]:
model = RandomForestClassifier()

# param_grid = {
#     'n_neighbors': [3, 5, 7, 9],
#     'weights': ['uniform', 'distance'],
#     'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
# }

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='roc_auc')
grid_search.fit(x, y)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Best Score: 0.6616983112883607


In [53]:
model = KNeighborsClassifier(n_neighbors=5, p=2, weights='distance')

In [55]:
model.fit(x,y)

KNeighborsClassifier(weights='distance')

In [57]:
x_test = df2.drop('flag', axis=1)
y_test = df2['flag']

In [58]:
y_pred_prob = model.predict_proba(x_test)[:, 1]

# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)

print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.5000392120700018


## результат
нихуя не работает:
ни одна из моделей не дала сколько нибудь интересной метрики за исключением:
1) KNN - при CV в рамках одного сета дает ахуевшие 0,8 без инжиниринга без нихуя, но на другом сете - 0,5 и отъебись - а я то уж обрадовался:((
2) лес дает скромные 0,65 - в теории если бы нашлась еще одна модель, которая хотя бы сносно что то предсказывала - можно было бы попробовать заставить их работать в паре

подробнее в блокноте

# 2 гиппотеза.  Берем последний продукт для каждого id

In [9]:
id_list = list(df.id.unique())

In [10]:
try:
    id_list = []
    with open('id_list_0.txt', 'r') as file:
        for line in file:
            id_list.append(
                (
                    eval(line.strip().split(', ')[0].replace('(','')), 
                    eval(line.strip().split(', ')[1].replace(')',''))
                )
            )
    
    
except FileNotFoundError:
    
    id_list = list(df.id.unique())
    for entry in range(len(id_list)):
        id_list[entry] =(id_list[entry], max(df[df.id == id_list[entry]].rn))

    with open('id_list_0.txt', 'w') as file:
        for item in id_list:
            file.write(str( item)+'\n')


In [68]:
df_cut = df[(df.id == id_list[0][0] )& (df.rn == id_list[0][1])]
for i in range(len(id_list)):
    new_row = df[(df.id == id_list[i][0])& (df.rn == id_list[i][1])]
    df_cut = pd.concat([df_cut, new_row], ignore_index=True)

In [72]:
df_cut = df_cut.drop(0, axis=0)

In [11]:
df_cut = pd.read_csv('train_data/df_only_last_product_0.csv')

In [74]:
df_cut.to_csv('train_data/df_only_last_product_0.csv', index=False)  

In [78]:
models = [
    KNeighborsClassifier(),
    SVC(),
    RandomForestClassifier(),
    MLPClassifier(),
    LogisticRegression(),
    DecisionTreeClassifier(),
    GaussianNB(),
]
#     xgb(),
#     lgb(),
# ]

In [11]:
x = df_cut.drop('flag', axis=1)
y = df_cut['flag']


In [12]:
results_dict = dict()

In [81]:
for model in models:
    
    cv_scores = cross_val_score(model, x, y, cv=3, scoring='roc_auc')
    results_dict[str(model)[0:str(model).find('(')]] = mean(cv_scores)


In [82]:
results_dict

{'KNeighborsClassifier': 0.5014837697443463,
 'SVC': 0.5041829161493093,
 'RandomForestClassifier': 0.6342966486892172,
 'MLPClassifier': 0.4885521343602927,
 'LogisticRegression': 0.5325920477964842,
 'DecisionTreeClassifier': 0.5332065228519628,
 'GaussianNB': 0.6265383537138974}

In [13]:
model = RandomForestClassifier()
model.fit(x, y)

KeyboardInterrupt: 

In [88]:
x_test = df2.drop('flag', axis=1)
y_test = df2['flag']
y_pred_prob = model.predict_proba(x_test)[:, 1]

# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)

print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.5835822646171664


In [90]:
# model = RandomForestClassifier()

# param_grid = {
#     'n_neighbors': [3, 5, 7, 9],
#     'weights': ['uniform', 'distance'],
#     'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
# }

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='roc_auc')
grid_search.fit(x, y)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Best Score: 0.6811226032781477


In [14]:
model = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_leaf=4, min_samples_split=10)
model.fit(x, y)

RandomForestClassifier(max_depth=20, min_samples_leaf=4, min_samples_split=10,
                       n_estimators=200)

In [15]:
x_test = df2.drop('flag', axis=1)
y_test = df2['flag']
y_pred_prob = model.predict_proba(x_test)[:, 1]

# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)

print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.6314349390953861


In [17]:
df2_id_rn = df2[['id','rn']]

In [20]:
id_list_1 = list(df2_id_rn.id.unique())

In [23]:
try:
    id_list_1 = []
    with open('id_list_1.txt', 'r') as file:
        for line in file:
            id_list_1.append(
                (
                    eval(line.strip().split(', ')[0].replace('(','')), 
                    eval(line.strip().split(', ')[1].replace(')',''))
                )
            )
    
    
except FileNotFoundError:
    
    id_list_1 = list(df2_id_rn.id.unique())
    for entry in range(len(id_list_1)):
        id_list_1[entry] =(id_list_1[entry], max(df2_id_rn[df2_id_rn.id == id_list_1[entry]].rn))

    with open('id_list_1.txt', 'w') as file:
        for item in id_list_1:
            file.write(str( item)+'\n')


In [24]:
df_cut_1 = df2[(df2.id == id_list_1[0][0] )& (df2.rn == id_list_1[0][1])]
for i in range(len(id_list_1)):
    new_row = df2[(df2.id == id_list_1[i][0])& (df2.rn == id_list_1[i][1])]
    df_cut_1 = pd.concat([df_cut_1, new_row], ignore_index=True)
df_cut_1.to_csv('train_data/df_only_last_product_1.csv', index=False)  

In [15]:
df_cut_1 = pd.read_csv('train_data/df_only_last_product_1.csv')
df_cut_1 = df_cut_1.drop(0, axis=0)

In [16]:
df_cut_1

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,flag
1,250000,7,11,9,4,8,1,11,0,5,...,3,3,4,1,2,3,1,1,1,0
2,250001,11,7,5,6,1,0,4,15,3,...,3,3,4,1,2,0,1,0,0,0
3,250002,3,9,6,4,8,1,11,1,1,...,3,3,4,1,2,3,1,1,1,0
4,250003,2,10,12,4,13,1,7,14,2,...,3,3,4,1,3,1,1,1,0,0
5,250004,2,19,0,1,8,14,11,8,2,...,3,3,4,1,2,4,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249996,499995,3,19,9,4,8,1,11,7,0,...,3,3,4,1,2,3,1,1,1,0
249997,499996,9,19,0,14,8,10,11,15,5,...,3,3,4,1,2,4,1,0,1,0
249998,499997,9,9,13,15,12,1,11,12,1,...,3,3,4,1,2,4,1,0,0,0
249999,499998,9,7,9,4,8,1,11,6,6,...,3,3,4,1,2,4,1,1,1,1


## результат 
ничего выдающегося
1) после грида валидация леса на новых данных дает 0,63
2) близко к лесу первичный поиск дает только некий gaussianDB но я не знаю что это такое, по этому пока не влезал - в конце концов метрика меньше, надеюсь дело не в рандомном разлете и я не пропустил сейчас выйгрышный билет
3) кажется придется все же разобраться с бустингом и/или рано или поздно прикрутить голосование
4) пидорская млп не работает - я помню так она меня выручила в первой курсовой:(

#  3 гиппотеза.  взять последний продукт для каждого id, смерджить пару сетов, сбалансировать, т.к. иначе очень большая получится

In [17]:
df_cut_merged = pd.concat([df_cut, df_cut_1], ignore_index=True)
df_cut_merged = df_cut_merged.drop(['id'], axis=1)

In [28]:
df_cut_merged = sample_df(df_cut_merged,df_cut_merged.flag.value_counts()[1], 50,50 )

sample_df3 start
sample_df3 end
-


In [30]:
x = df_cut_merged.drop('flag', axis=1)
y = df_cut_merged.flag

## а теперь все вместе, чтобы можно было оставить на полдня:)

In [56]:
# создаем список для результатов
results_list = []

In [ ]:
# задать варианты балансировки - 50%, 45% положительных и т.д.
shares = [50, 45, 40 ,35, 30, 25, 20, 15, 10, 5]

for i in shares:
    results_dict = dict()
    df_cut_merged = pd.concat([df_cut, df_cut_1], ignore_index=True)
    df_cut_merged = df_cut_merged.drop(['id'], axis=1)
    df_cut_merged = sample_df(
        df_cut_merged, 
        round(df_cut_merged.flag.value_counts()[1] / i * 100 + 1, 0),
        100 - i,
        i
    )
    
    # пройтись по всем моделям и записать результаты в словарь
    models = [
        KNeighborsClassifier(),
        SVC(),
        RandomForestClassifier(),
        MLPClassifier(),
        LogisticRegression(),
        DecisionTreeClassifier(),
        GaussianNB(),
    ]
    #     xgb(),
    #     lgb(),
    # ]
    x = df_cut_merged.drop('flag', axis=1)
    y = df_cut_merged.flag
    
    for model in models:
    
        cv_scores = cross_val_score(model, x, y, cv=3, scoring='roc_auc')
        results_dict[str(model)[0:str(model).find('(')]] = mean(cv_scores)


    # и добавить словарь в список словарей с результатами вместе с описанием доли
    results_list.append(f'share is {i}')
    results_list.append(results_dict)
    print('share ', i,' is done')
    
    
    
#     print('share is ',i, ', df size is ', df_cut_merged.shape, ' pos flag is ')

In [ ]:
results_list

## параллельно попробуем запустить разовое обучение на сете 50/50 и опробуем на третьем сете

In [12]:
df_cut = pd.read_csv('train_data/df_only_last_product_0.csv')
df_cut_1 = pd.read_csv('train_data/df_only_last_product_1.csv')
df_cut_merged = pd.concat([df_cut, df_cut_1], ignore_index=True)
df_cut_merged = df_cut_merged.drop(['id'], axis=1)
df_cut_merged = sample_df(df_cut_merged, df_cut_merged.flag.value_counts()[1] * 2, 50, 50)

In [14]:
x = df_cut_merged.drop('flag', axis=1)
y = df_cut_merged.flag

In [15]:
models = [
        KNeighborsClassifier(),
        SVC(),
        RandomForestClassifier(),
        MLPClassifier(),
        LogisticRegression(),
        DecisionTreeClassifier(),
        GaussianNB(),
    ]

In [16]:
results_dict = dict()

for model in models:

    cv_scores = cross_val_score(model, x, y, cv=3, scoring='roc_auc')
    results_dict[str(model)[0:str(model).find('(')]] = mean(cv_scores)

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_i

In [17]:
results_dict

{'KNeighborsClassifier': 0.6082156920067399,
 'SVC': 0.680661354978124,
 'RandomForestClassifier': 0.6853091487422303,
 'MLPClassifier': 0.666758333025849,
 'LogisticRegression': 0.6783188926211895,
 'DecisionTreeClassifier': 0.5657900367690957,
 'GaussianNB': 0.6506747711100201}

In [41]:
models = [
#         KNeighborsClassifier(),
        SVC(probability=True),
        RandomForestClassifier(),
        MLPClassifier(),
        LogisticRegression(),
#         DecisionTreeClassifier(),
#         GaussianNB(),
    ]

x = df_cut_merged.drop('flag', axis=1)
y = df_cut_merged.flag

In [42]:
results_dict_2_stage = dict()

In [43]:
df3 = open_df(2)
df3 = pd.merge(df3, answers, on='id', how='left')
df3 = df3.drop('id', axis=1)


In [44]:
x_test = df3.drop('flag', axis=1)
y_test = df3['flag']

In [45]:
for model in models:
    model.fit(x,y)
    y_pred_prob = model.predict_proba(x_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    
    results_dict_2_stage[str(model)[0:str(model).find('(')]] = roc_auc

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [46]:
results_dict_2_stage

{'SVC': 0.6126069032681178,
 'RandomForestClassifier': 0.6246506905604878,
 'MLPClassifier': 0.6034057912225304,
 'LogisticRegression': 0.6119021451189127}

model = KNeighborsClassifier(n_neighbors=5, p=2, weights='distance')
model.fit(x,y)
x_test = df2.drop('flag', axis=1)
y_test = df2['flag']
y_pred_prob = model.predict_proba(x_test)[:, 1]

# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)

print("ROC AUC Score:", roc_auc)

In [15]:
new_row = dict()
for col in cols:
    new_row[col] = list(df.loc[df['id'] == 0,col])


In [16]:
new_row_df = pd.DataFrame.from_records(new_row)

new_row_df.iloc[1,1] = [1,2]

In [17]:
answers

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2999995,2999995,0
2999996,2999996,0
2999997,2999997,0
2999998,2999998,0


sns.heatmap(df_balanced.corr(), cmap='coolwarm', linewidths=0.5)


df_corr = df_balanced.corr()

columns = df_corr.columns
rows = df_corr.columns

results_acc = [[]]*len(threshholds)

results_acc

In [100]:
threshholds = [0.01, 0.05, 0.08]#,[0.02, 0.03, 0.04, 0.05, 0.06, 0.07]
results_acc = [[]]*len(threshholds)
results_roc = [[]]*len(threshholds)
counter = 0
for entry_1 in threshholds:
    
    print('threshhold is ', entry_1)

    corrs = []
    for col in columns:
        corrs.append((df_balanced.flag.corr(df_balanced[col]), col))
    corrs = corrs[:-1]
    corrs = sorted(corrs)


    significant_cols = []

    threshhold = entry_1

    for entry in corrs:
        if entry[0] < -threshhold or entry[0] > threshhold:
            significant_cols.append(entry[1])
    significant_cols.append('flag')
    
    
    df_balanced_cut = df_balanced[significant_cols]
    
    for i in range(3):
        
        df_train, df_test = train_test_split(df_balanced_cut, stratify=df_balanced_cut['flag'], test_size=0.3)

        x_train = df_train.drop('flag', axis=1)
        y_train = df_train.flag
        x_test = df_test.drop('flag', axis=1)
        y_test = df_test.flag

        model = RandomForestClassifier()
        model.fit(x_train, y_train)

        
        pred_train = model.predict(x_train)
        pred_test = model.predict(x_test)
    #     print('train score acc - ', accuracy_score(y_train, pred_train))
    #     print('test score acc - ', accuracy_score(y_test, pred_test))
        results_acc[counter].append(accuracy_score(y_test, pred_test))


        prob_train = model.predict_proba(x_train)[:, 1]
        prob_test = model.predict_proba(x_test)[:, 1]
    #     print('train score roc - ', roc_auc_score(y_train, prob_train))
    #     print('test score roc - ', roc_auc_score(y_test, prob_test))
        results_roc[counter].append(roc_auc_score(y_test, prob_test))
        
        print(i, ' fits done in loop ', counter)



    counter += 1

threshhold is  0.01
0  fits done in loop  0
1  fits done in loop  0
2  fits done in loop  0
threshhold is  0.05
0  fits done in loop  1
1  fits done in loop  1
2  fits done in loop  1
threshhold is  0.08
0  fits done in loop  2
1  fits done in loop  2
2  fits done in loop  2


In [106]:
results_roc

[[0.6377428977777777,
  0.6372906488888889,
  0.6340239244444446,
  0.609586251111111,
  0.6111367733333335,
  0.6120779888888889,
  0.60261152,
  0.5999410177777779,
  0.6023576733333333],
 [0.6377428977777777,
  0.6372906488888889,
  0.6340239244444446,
  0.609586251111111,
  0.6111367733333335,
  0.6120779888888889,
  0.60261152,
  0.5999410177777779,
  0.6023576733333333],
 [0.6377428977777777,
  0.6372906488888889,
  0.6340239244444446,
  0.609586251111111,
  0.6111367733333335,
  0.6120779888888889,
  0.60261152,
  0.5999410177777779,
  0.6023576733333333]]

for i in range(len(results_roc)):
    print('for threshhold ',threshholds[i], ' avg is ', results_roc[i].mean())
for i in range(len(results_roc)):
    print('for threshhold ',threshholds[i], ' avg is ', results_acc[i].mean())

In [84]:
df_train, df_test = train_test_split(df_balanced, stratify=df_balanced['flag'], test_size=0.3)

In [85]:
x_train = df_train.drop('flag', axis=1)
y_train = df_train.flag
x_test = df_test.drop('flag', axis=1)
y_test = df_test.flag

rf =  RandomForestClassifier()

rf.fit(x_train, y_train)

In [49]:
model = LogisticRegression()

In [86]:
model = RandomForestClassifier()

In [59]:
model  = MLPClassifier()

# готовый скрипт 

In [75]:
# путь до данных на компьютере
path = 'train_data/'

In [48]:
import os
import pandas as pd
import tqdm


def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                     num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
    """
    читает num_parts_to_read партиций, преобразовывает их к pd.DataFrame и возвращает
    :param path_to_dataset: путь до директории с партициями
    :param start_from: номер партиции, с которой нужно начать чтение
    :param num_parts_to_read: количество партиций, которые требуется прочитать
    :param columns: список колонок, которые нужно прочитать из партиции
    :return: pd.DataFrame
    """

    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                              if filename.startswith('train')])
    print(dataset_paths)

    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        print('chunk_path', chunk_path)
        chunk = pd.read_parquet(chunk_path,columns=columns)
        res.append(chunk)

    return pd.concat(res).reset_index(drop=True)

In [50]:
# answers
check = []
for i in range(len(answers)-1):
#     if i == 
    check.append(answers.id[i]+1 == answers.id[i+1])